In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import pandas as pd
import copy

In [2]:
Cave = np.loadtxt('Cave.csv')
pop = np.loadtxt('pop_new.csv')

In [5]:
num_fips = len(pop)
T = 60
num_ens = 300

In [13]:
# compute wji make the transission rate(probability)
# mobility factor
theta = 1
W = np.zeros_like(Cave)
WN = np.zeros_like(Cave)
for i in range(num_fips):
    non_dia = np.delete(idx, i)
    W[non_dia,i] = Cave[non_dia,i]/pop[i] 
    W[i,i] = max((1-np.sum(W[non_dia,i])),0.2) ## the maxium probability travel within is 0.2
    WN[:,i] = W[:,i]/np.sum(W[:,i]) ## normalise distribution

In [62]:
np.savetxt('W_avg.csv',WN)

In [41]:
# pathogen characteristics
# initialize parameters
R0 = 2.5 
r = 0.1 ## the parameters
Z = 3 # latent period
Zb = 1 # scale parameter for Z
D = 5 # infectious period
Db = 1 # scale parameter for b
alpha = 0.1 # reporting rate 10%
# R0,r,Z;Zb;D;Db,alpha
para_i = np.array([[R0], [r], [Z], [Zb], [D], [Db], [alpha]])

In [47]:
para = np.repeat(para_i,300,axis=1)

In [60]:
#initialize variables

NewInf = np.zeros((num_fips,T,num_ens))
Obs = np.zeros((num_fips,T,num_ens))
# seeding
l0 = 1859-1 # start with New York County NY in python -1, in matlab is 1895
NewInf[l0,0,:] = 100 ## the starting t=0, in matlab it is 1

In [ ]:
%%%%%%%%%%%%%%%%%%%%%%%%%%%%
for t=1:T
    t
    tic
    for k=1:num_ens

        %%%%%%%%%%%%%%%%%%%%%%%%%
        NewInf(:,:,k)=branching(nl,part,w,t,NewInf(:,:,k),para(:,k),population);
    end
    toc
end

In [ ]:
Obs=alpha*NewInf;
% if r==0.1
%     subplot(1,2,1)
%     title('r=0.1')
% % elseif r==0.3
% %     subplot(1,3,2)
% %     title('r=0.3')
% elseif r==5
%     subplot(1,2,2)
%     title('r=5')
% end
DailyInf=squeeze(sum(NewInf,1));
hold on
plot(1:T,DailyInf)
plot(1:T,median(DailyInf,2),'k-','LineWidth',2),
ylabel('Daily Infection')

% if r==0.1
%     save NewInf_01.mat NewInf Obs
% elseif r==0.3
%     save NewInf_03.mat NewInf Obs
% elseif r==5
%     save NewInf_5.mat NewInf Obs
% end
